In [317]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('../partitions/train.csv', sep=';')
df_test = pd.read_csv('../partitions/test.csv', sep=';')

matriz_datos_train = feature_extraction(df_train)
matriz_datos_test = feature_extraction(df_test)

# print(np.shape(matriz_datos_test))

import os
if not os.path.exists('../features'):
    os.mkdir('../features')

np.save('../features/matriz_datos_train.npy', matriz_datos_train)
np.save('../features/matriz_datos_test.npy', matriz_datos_test)

(320, 17)
(80, 17)


In [315]:
#En esta función extraemos las características necesarias para realizar el modelado
def feature_extraction(df):
    import cv2
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import pandas as pd
    import numpy as np
    from re import search
    from skimage.measure import label
    from skimage.morphology import convex_hull_image
    from skimage.feature import greycomatrix, greycoprops
    import math
    #Vamos a coger la base de datos de train

    fingerprint=[]
    for i in range(0,len(df)):
    
        global marca,erosion
       #el siguiente clasifica las imágenes según su base de datos, para buscar la imagen en la carpeta que se encuentra
        file = df.ID[i]
        if search("_2",file):
            img_original = cv2.imread('../sin_marca_agua/'+ file)
            img_gray_1 = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
            _, binary_1 = cv2.threshold(img_gray_1,0,1,cv2.THRESH_OTSU)
            inverted_image_1 = np.invert(binary_1)
        else:
      
            marca = cv2.imread('../con_marca_agua/' + file)
            img_gray_2 = cv2.cvtColor(marca, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(img_gray_2,0,1,cv2.THRESH_OTSU)
            imagen_resultante=np.invert(binary) #onvertimos la binarización
            kernel=np.ones((3,3),np.uint8)#aplicamos un kernel para seleccionar en la erosión
            erosion=cv2.erode(imagen_resultante,kernel)#utilizamos la erosión para no juntar los pixeles blancos
           
          
    
    
    
        #Descriptores de textura
        #Extraemos descriptores de textura GLCM
        GLCM = greycomatrix(erosion, distances=[2], angles=[90], levels=256, symmetric=True, normed=True)
        contraste = greycoprops(GLCM, 'contrast')[0,0]
        disimilitud = greycoprops(GLCM, 'dissimilarity')[0,0]
        homogeneidad = greycoprops(GLCM, 'homogeneity')[0,0]
        ASM = greycoprops(GLCM, 'ASM')[0,0]
        energia = greycoprops(GLCM, 'energy')[0,0]
        correlacion = greycoprops(GLCM, 'correlation')[0,0]
        # Local Binary Patterns (LBP)
        from skimage.feature import local_binary_pattern
        R=1 # radio
        P=8*R # vecinos
        lbp_image = local_binary_pattern(erosion, P, R, method='uniform')
            
        lbp_image = np.uint8(lbp_image)
            
        hist_lbp = cv2.calcHist([lbp_image.ravel()], [0], None, [P+2], [0, P+2])
        hist_lbp = hist_lbp.astype('float')
        hist_lbp /= (hist_lbp.sum() + 1e-7)
        hist_lbp = hist_lbp.tolist()
        hist_lbp = [item for sublist in hist_lbp for item in sublist]
    
        #plt.imshow(erosion, cmap='gray')
        #plt.show()
        
        #plt.imshow(lbp_image, cmap='gray')
        #plt.show()
        
        #plt.plot(hist_lbp)
        #plt.grid(True)
        #plt.show()
    
        features_textura = [contraste, disimilitud,homogeneidad, ASM, energia, correlacion] + hist_lbp # Características bidimensionales
            
        # Extraer la información de la clase
        if df.Class[i] == 'Marca':
            etiqueta = [1]
        else:
            etiqueta = [0]
        
        fingerprint.append(features_textura + etiqueta)
    #    print(np.shape(fingerprint))
    
    matriz_datos = np.array(fingerprint)
    print(np.shape(matriz_datos))
    return matriz_datos